In [ ]:
import copy
import math
import matplotlib.pyplot as plt

class Polynomial:

  def __init__ (self, polynomial = {}):
    
    self.dictionary = {}
    self.coefficients_with_repetition = []
    degree = 0

    if isinstance(polynomial, str):

      modified_polynomial = polynomial.replace('-', ' -').replace('+', ' ')
      polynomial_terms = modified_polynomial.split()
      self.dictionary = {}

      for term in polynomial_terms: # We split the term into coefficient and exponent, taking out the ^ character.

        if 'x' in term:
          (coefficient, exponent) = term.split('x')
          if coefficient == '':
            coefficient = 1.0
          elif coefficient == '-':
            coefficient = -1.0
          else:
            coefficient = float(coefficient)
          if exponent == '':
            exponent = 1
          else:
            exponent = int(exponent [1:])

        else:
          (coefficient, exponent) = (float(term), 0)
        
        if coefficient != 0.0:
          self.dictionary[exponent] = coefficient
          self.coefficients_with_repetition.append(coefficient)
          if exponent > degree:
            degree = exponent
    
    elif isinstance(polynomial, dict):
      for exponent in polynomial:
        coefficient = float(polynomial[exponent])
        exponent = int(exponent)
        if coefficient != 0.0:
          self.dictionary[exponent] = coefficient
          self.coefficients_with_repetition.append(coefficient)
          if exponent > degree:
            degree = exponent
    
    elif isinstance(polynomial, int):
      coefficient = float(polynomial)
      if coefficient != 0.0:
        self.dictionary[0] = float(polynomial)
        self.coefficients_with_repetition.append(coefficient)
        degree = 1
  
    elif isinstance(polynomial, float):
      if polynomial != 0.0:
        self.dictionary[0] = polynomial
        self.coefficients.add(polynomial)
        self.coefficients_with_repetition.append(coefficient)
        degree = 1

    else:
      raise TypeError(f'The polynomial must be initialized using a string, a dictionary, an integer, a float, or no argument (which results in a zero polynomial).\nReceived type: {type(polynomial).__name__}.')
    
    self.coefficients = set(self.coefficients_with_repetition)
    self.degree = degree
    # Note that we will have an empty dictionary for the zero polynomial.
    self.leading_coefficient = self.dictionary.get(degree, 0.0)
    if self.leading_coefficient > 0.0:
      self.leading_coefficient_sign = '+'
    elif self.leading_coefficient < 0.0:
      self.leading_coefficient_sign = '-'
    else:
      self.leading_coefficient_sign = 'zero'
    
    # This tracks if our polynomial is the result of an indefinite integral.
    self.is_integral_result = False

  def __str__ (self) -> str:

    sorted_dictionary_items = sorted(self.dictionary.items(), reverse=True)
    dictionary_string = ''

    for (exponent, coefficient) in sorted_dictionary_items:
 
      # Note that we can't have a 0.0 coefficient by the way we constructed self.dictionary, but we handle it in case the user directly modifies self.dictionary.

      if coefficient == 0.0:
        continue

      signed_coefficient = '+' if coefficient > 0.0 else '-'
      dictionary_string += signed_coefficient
      
      if (abs(coefficient) != 1.0 and exponent > 0) or (exponent == 0): # Except for the constant term, 1 and -1 coefficients are represented as + and - in a polynomial.
        dictionary_string += str(abs(coefficient))
      
      if exponent > 1:
        dictionary_string += f'x^{exponent}'

      elif exponent == 1:
        dictionary_string += 'x'

    if dictionary_string[0] == '+':
      dictionary_string = dictionary_string[1:]

    if self.is_integral_result:
      dictionary_string += '+C'
      
    if dictionary_string == '':
      dictionary_string = '0.0'

    return dictionary_string
  
  def evaluate (self, x) -> float:
    if self.is_integral_result:
       raise ValueError("The evaluate method doesn't support a polynomial that is the result of an indefinite integral. Try evaluate_with_constant instead.")
    result = 0.0
    for (exponent, coefficient) in self.dictionary.items():
      result += coefficient*math.pow(x, exponent)
    return result
  
  def evaluate_with_constant (self, x) -> str:
    if not self.is_integral_result:
      raise ValueError("The evaluate_with_constant method doesn't support a polynomial that isn't the result of an indefinite integral. Try evaluate instead.")
    polynomial_without_constant = copy.copy(self)
    polynomial_without_constant.is_integral_result = False
    result = str(polynomial_without_constant.evaluate(x))
    result += '+C'
    return result
  
  def sign_behavior (self, x, epsilon = 1e-10) -> str:
    # We use an epsilon as we are working with floating numbers, which may produce an error.
    if self.is_integral_result:
      raise ValueError('The sign is not determined because of the constant of integration.')
    if abs(self.evaluate(x)) <= epsilon:
      return 'zero'
    elif self.evaluate(x) > 0.0:
      return '+'
    else:
      return '-'

  def sign_behavior_at_infinity (self) -> tuple:
    if self.degree == 0 and self.is_integral_result:
      # In a constant polynomial resulting of an indefinite integral, it isn't meaningful to look at the sign of a constant plus our indefinite integral constant.
      raise ValueError('The sign at infinity is not determined in a constant function plus the constant of integration.')
      return (('+', self.evaluate(0)), ('-', self.evaluate(0)))
    elif self.degree % 2 == 0:
      if self.leading_coefficient_sign == '+':
        return (('+', float('inf')), ('-', float('inf')))
      elif self.leading_coefficient_sign == '-':
        return (('+', float('-inf')), ('-', float('-inf')))
    else:
      if self.leading_coefficient_sign == '+':
        return (('+', float('inf')), ('-', float('-inf')))
      else:
        return (('+', float('-inf')), ('-', float('inf')))

  def is_monic (self) -> bool:
    return self.leading_coefficient == 1 

  def convert_to_monic (self) -> Polynomial:
    if self.leading_coefficient == 0.0:
      raise ValueError('Zero polynomials cannot be converted to monic form!')
    elif self.leading_coefficient == 1.0:
      return self
    else:
      monic_polynomial_dict = {}
      for exponent, coefficient in self.dictionary.items():
        monic_polynomial_dict[exponent] = coefficient / self.leading_coefficient
      return Polynomial(monic_polynomial_dict)
  
  def is_constant (self) -> bool:
    return self.degree == 0

  def is_zero_polynomial (self) -> bool:
    return self.degree == 0 and self.leading_coefficient == 0.0 and not self.is_integral_result
  
  def is_root (self, value, epsilon=1e-10) -> bool:
    return abs(self.evaluate(value)) < epsilon

  def derive (self) -> Polynomial:
    derivative_dictionary = {}
    for exponent, coefficient in self.dictionary.items():
      if exponent > 0:
        derivative_dictionary[exponent-1] = coefficient*exponent
    # Note that the resulting derivative is_integral_result attribute is False.
    return Polynomial(derivative_dictionary)
  
  def compute_n_th_derivative (self, n: int) -> Polynomial:
    if not isinstance(n, int) or n < 0:
      raise ValueError(f'The argument must be a non negative integer.\nReceived value: {n}.')
    n_th_derivative_dict = {}
    for exponent, coefficient in self.dictionary.items():
      if exponent >= n:
        # Note that deriving n times is the same as reducing each exponent by n and multiplying its corresponding coefficient by exponent, then exponent - 1, ..., then exponent - n + 1.
        n_th_derivative_dict[exponent-n] = (math.factorial(exponent) // math.factorial(exponent-n))*coefficient
    return Polynomial(n_th_derivative_dict)
  
  def antiderive (self) -> Polynomial:
    if self.is_integral_result == True:
      raise ValueError('Polynomial is already an indefinite integral result. Further integration is not supported.')
    antiderivative_dict = {}
    for exponent, coefficient in self.dictionary.items():
      antiderivative_dict[exponent+1] = coefficient/(exponent+1)
    antiderivative = Polynomial(antiderivative_dict)
    antiderivative.is_integral_result = True
    return antiderivative
  
  def evaluate_derivative (self, x) -> float:
    derivative = self.derive()
    return derivative.evaluate(x)
  
  def evaluate_n_th_derivative (self, n: int, x) -> float:
    n_th_derivative = self.compute_n_th_derivative(n)
    return n_th_derivative.evaluate(x)
  
  def definite_integral (self, a, b) -> float:
    an_antiderivative = self.antiderive()
    an_antiderivative.is_integral_result = False
    return an_antiderivative.evaluate(b) - an_antiderivative.evaluate(a)
  
  def get_degree (self):
    return self.degree
  
  def get_coefficients (self, *exponents) -> tuple:    
    desired_exponents_coefficients = ()
    for exponent in exponents:
      desired_exponents_coefficients += (exponent, self.dictionary.get(exponent, 0.0))
    return desired_exponents_coefficients
  
  def set_coefficients (self, exponents, coefficients):

    if len(exponents) != len(coefficients):
      raise ValueError(f'We must have the same amount of exponents and their associated coefficients.\nNumber of exponents received: {len(exponents)}.\nNumber of coefficients received: {len(coefficients)}.')
    exponents_and_coefficients = zip(exponents, coefficients)
    for exponent, coefficient in exponents_and_coefficients:

      if exponent < 0 or not isinstance(exponent, int):
        raise TypeError(f'Exponents must be non negative integers.\nInvalid exponent used: {exponent} and its type {type(exponent).__name__}.')
      if not (isinstance(coefficient, int) or isinstance(coefficient, float)):
        raise TypeError(f'Coefficients must be numbers: integers or floats.\nInvalid coefficient used: {coefficient} and its type {type(coefficient).__name__}.')
      if isinstance(coefficient, int):
        coefficient = float(coefficient)

      if exponent in self.dictionary:
        for index, old_coefficient in enumerate(self.coefficients_with_repetition):
          if old_coefficient == self.dictionary[exponent]:
            break

        if coefficient != 0.0:
          self.coefficients_with_repetition[index] = coefficient
          if exponent == self.degree:
            self.leading_coefficient = coefficient
            if coefficient > 0.0:
              self.leading_coefficient_sign = '+'
            elif coefficient < 0.0:
              self.leading_coefficient_sign = '-'
            else:
              self.leading_coefficient_sign = 'zero'

        else:
          del self.dictionary[exponent]
          self.coefficients_with_repetition.pop(index)
          if exponent == self.degree:
            highest_exponent = 0
            for power in self.dictionary:
              if power > highest_exponent:
                highest_exponent = power
            self.degree = highest_exponent

        self.coefficients = set(self.coefficients_with_repetition)
      
      else:
        # We do nothing if the coefficient is zero and its corresponding exponent is not in the polynomial.
        if coefficient != 0.0:
          self.dictionary[exponent] = coefficient
          self.coefficients_with_repetition.append(coefficient)
          self.coefficients.add(coefficient)
      
      # Note that, if all the coefficients were replaced by zeros, we will not have a corresponding value to the key self.degree.
      self.leading_coefficient = self.dictionary.get(self.degree, 0.0)
      if self.leading_coefficient > 0.0:
        self.leading_coefficient_sign = '+'
      elif self.leading_coefficient < 0.0
        self.leading_coefficient_sign = '-'
      else:
        self.leading_coefficient_sign = 'zero'

    return Polynomial(self.dictionary)
  def __add__ (self, other):
    pass

pol = Polynomial({1: 1, 0: -1})

print(pol)

print(enumerate([1,2,3,4,6]))

for i, j in enumerate([1,2,3,5]):
  print(i,j)

x-1.0
0 1
1 2
2 3
3 5


In [ ]:
"""
Coisas para melhorar:
- Representação em str com espaços e aceitar mais representações de str;
- Calcular numericamente comprimento de curva;
- Calcular numericamente derivada, n-ésima derivada e integral definida;
- Adicionar os métodos mágicos: add, sub, mul, eq, repr, len, iter (daí melhorar alguns códigos em que eu itero nos itens), getitem, setitem, call, talvez copy;
- Adicionar métodos get_coefficient, set_coefficient, get_terms (tupla de tuplas de expoente e coeficiente), is_root;
- Otimizar algumas coisas;
- Escrever de forma mais limpa;
- Representação em gráfico (derivada, n-ésima derivada, integral definida)
- Escrever um readme;
- Abrir arquivos;
- Métodos de Newton e da bissecção!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
- Revisar.
"""


'\nCoisas para melhorar:\n- Representação em str com espaços e aceitar mais representações de str;\n- Calcular numericamente comprimento de curva;\n- Calcular numericamente derivada, n-ésima derivada e integral definida;\n- Adicionar os métodos mágicos: add, sub, mul, eq, repr, len, iter (daí melhorar alguns códigos em que eu itero nos itens), getitem, setitem, call, talvez copy;\n- Adicionar métodos get_coefficient, set_coefficient, get_terms (tupla de tuplas de expoente e coeficiente), is_root;\n- Otimizar algumas coisas;\n- Escrever de forma mais limpa;\n- Representação em gráfico (derivada, n-ésima derivada, integral definida)\n- Escrever um readme;\n- Abrir arquivos;\n- Métodos de Newton e da bissecção!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n- Revisar.\n'